In [2]:
from kmodes.kmodes import KModes

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib.pyplot import xticks

In [6]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options

from bs4 import BeautifulSoup

In [7]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")

In [8]:
#!pip install webdriver_manager --user

In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

#browser = webdriver.Chrome("chromedriver.exe", options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/home/carlos.cayres/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver] found in cache


In [ ]:
browser.get("https://clutch.co/it-services/outsourcing")

In [ ]:
# image = browser.find_element_by_class_name('company_logotype') #class="company_logotype"
# image.click()
soup = BeautifulSoup(browser.page_source, features='html.parser')
url_list = soup.find_all("div",attrs={"class" : "company_logotype"}, recursive=True)

In [ ]:
# nome -- h1 class header-company--title 
# nota -- span itemprop ratingValue
# local -- span class location-name
# clients -- div class field-name-clients // field c -> p
browser.get("https://clutch.co/profile/pattern-match")

In [ ]:
url_list_string = []
for i in range(2):
    browser.get(f"https://clutch.co/it-services/outsourcing?page={i}")
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    #url_list.append(soup.find_all("div",attrs={"class" : "company_logotype"}, recursive=True))
    url_list = (soup.find_all("div",attrs={"class" : "company_logotype"}, recursive=True))
    
    for url in url_list:
           url_list_string.append(url.find('a')['href'])

In [ ]:
name = []
note = []
place = []
client = []
value = []
hour_price = []
employee = []

for url in url_list_string:
    #print(url.find('a')['href'])
    browser.get(f"https://clutch.co{url}")
    time.sleep(3)
    soup = BeautifulSoup(browser.page_source, features='html.parser')

    nome = soup.find("h1", attrs={"class" : "header-company--title"})
    nome = nome.text.replace('\n', '').strip()
    name.append(nome)
    
    main_div = soup.find("div", attrs={"class" : "module-list"}) 
    main_div.find_all("span")
    
    valor = ''
    preco_hora = ''
    funcionarios = ''
    
    main_div = [i.text for i in main_div.find_all("span")]
    main_div
    valor, preco_hora, funcionarios = [main_div[i] for i in (0, 1, 2)]
    value.append(valor)
    hour_price.append(preco_hora)
    employee.append(funcionarios)
    

    nota = soup.find("span", attrs={"itemprop" : "ratingValue"})
    nota = nota.text.replace('\n', '').strip()
    note.append(nota)

    local = soup.find("span", attrs={"class" : "location-name"})
    local = local.text.replace('\n', '').strip()
    place.append(local)
    
    clients = ''
    try:
        clients = soup.find("div", attrs={"class" : "field-name-clients"})
        clients =  ' '.join([p.text.replace('\n', ' ').strip() for p in clients.find_all('p')])
        client.append(clients)
    except:
        try:
            clients = soup.find("div", attrs={"class" : "field c"})
            clients = ' '.join([p.text.replace('\n', ' ').strip() for p in clients.find_all('p')])
            #clients = clients.find('p').text.replace('\n', ' ').strip()
            client.append(clients)
        except:
            clients = ' : '.join(["Clientes não encontrados para a empresa",nome])
            clients = f"Clientes não encontrados para a empresa : {nome}"

            client.append(clients)
   #print(f"nome: {nome}")
   #print(f"nota: {nota}")
   #print(f"local: {local}")
   #print(f"clientes: {clients}")
   #print(f"valor : {valor}")
   #print(f"preco_hora : {preco_hora}")
   #print(f"funcionarios : {funcionarios}")
print(f"FIM")

In [ ]:
def preenche_pais(x):
    if len(x.split(", ")[1]) == 2:
        return (x.split(", ")[1]), "United States"
    else:
        return " ", x.split(", ")[1]
    

In [ ]:
df = pd.DataFrame({'nome': name, 'nota': note, 'valor':value, 'preco_hora':hour_price, 'funcionarios':employee, 'local': place, 'clients':client})

In [ ]:
df['cidade'] = df.local.str.split(",",expand=True,)[0]
df[['estado','pais']] = pd.DataFrame(df.local.apply(preenche_pais).tolist(), index=df.index)
df.head()

In [ ]:
df_model = df[['nota','valor','preco_hora','funcionarios']].copy()

In [ ]:
df_model['nota'] = df['nota'].apply(lambda x : float(x))

In [ ]:
df_model['faixa_notas'] = pd.cut((df_model.nota),2)
df_model.drop(columns=["nota"],inplace=True)

In [ ]:
df_model.isnull().sum()*100/df_model.shape[0]

In [ ]:
km = KModes(n_clusters=3, init='Huang', n_init=5, verbose=1)

clusters = km.fit_predict(df_model)

df_model['cluster'] = clusters

# Print the cluster centroids
print(km.cluster_centroids_)

In [ ]:
plt.subplots(figsize = (15,5))
sns.countplot(x=df_model['valor'],order=df_model['valor'].value_counts().index,hue=df_model['cluster'])
plt.show()

In [ ]:
plt.subplots(figsize = (15,5))
sns.countplot(x=df_model['funcionarios'],order=df_model['funcionarios'].value_counts().index,hue=df_model['cluster'])
plt.show()

In [ ]:
plt.subplots(figsize = (15,5))
sns.countplot(x=df_model['preco_hora'],order=df_model['preco_hora'].value_counts().index,hue=df_model['cluster'])
plt.show()

In [ ]:
plt.subplots(figsize = (15,5))
sns.countplot(x=df_model['faixa_notas'],order=df_model['faixa_notas'].value_counts().index,hue=df_model['cluster'])
plt.show()